In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf 
from tensorflow.keras.models import load_model

In [2]:
# Load the saved model
model = load_model('model.keras')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 34, 34, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 73984)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │       4,735,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 7)                   │             455 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,791,815 (18.28 MB)

 Trainable params: 4,791,815 (18.28 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Define the GUI class
class TrashRecycleGUI(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Trash and Recyclable Material Identification")
        self.geometry("600x400")
        self.resizable(False, False)
        self.configure(bg="#f0f0f0")
        self.create_widgets()
        self.image_path = None
        self.image = None
        self.predicted_class = None
        self.predicted_probability = None
        self.class_names = ["cardboard", "compost", "glass", "metal", "paper", "plastic", "trash"]
        self.class_labels = ["Cardboard", "Compost", "Glass", "Metal", "Paper", "Plastic", "Trash"]
        self.class_colors = ["#ff0000", "#0fff00", "#00ff00", "#0000ff", "#ffff00", "#ff00ff", "#00ffff"]
        self.class_colors = [tuple(int(color.lstrip("#")[i:i+2], 16) / 255 for i in (0, 2, 4)) for color in self.class_colors]
        
    def create_widgets(self):
        # Create a frame to hold the image
        self.image_frame = tk.Frame(self, bg="#f0f0f0")
        self.image_frame.pack(pady=10)
        
        # Create a label to display the image
        self.image_label = tk.Label(self.image_frame, bg="#f0f0f0")
        self.image_label.pack()
        
        # Create a frame to hold the predicted class and probability
        self.predicted_frame = tk.Frame(self, bg="#f0f0f0")
        self.predicted_frame.pack(pady=10)
        
        # Create a label to display the predicted class
        self.predicted_class_label = tk.Label(self.predicted_frame, text="Predicted Class:", bg="#f0f0f0")
        self.predicted_class_label.pack(side="left", padx=10)
        
        # Create a label to display the predicted probability
        self.predicted_probability_label = tk.Label(self.predicted_frame, text="Predicted Probability:", bg="#f0f0f0")
        self.predicted_probability_label.pack(side="left", padx=10)
        
        # Create a label to display the predicted material
        self.predicted_material_label = tk.Label(self.predicted_frame, text="Predicted Material:", bg="#f0f0f0")
        self.predicted_material_label.pack(side="left", padx=10)
        
        # Create a frame to hold the buttons
        self.button_frame = tk.Frame(self, bg="#f0f0f0")
        self.button_frame.pack(pady=10)
        
        # Create a button to select an image
        self.select_button = tk.Button(self.button_frame, text="Select Image", command=self.select_image)
        self.select_button.pack(side="left", padx=10)
        
        # Create a button to classify the image
        self.classify_button = tk.Button(self.button_frame, text="Classify Image", command=self.classify_image)
        self.classify_button.pack(side="left", padx=10)
        
        # Create a button to clear the image and predicted class
        self.clear_button = tk.Button(self.button_frame, text="Clear", command=self.clear_image)
        self.clear_button.pack(side="left", padx=10)
        
    def select_image(self):
        # Open a file dialog to select an image
        self.image_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if self.image_path:
            # Load the image and resize it to 224x224 pixels
            self.image = Image.open(self.image_path)
            self.image = self.image.resize((224, 224))
            self.image = ImageTk.PhotoImage(self.image)
            
            # Update the image label with the selected image
            self.image_label.configure(image=self.image)
            self.image_label.image = self.image
            self.predicted_class = None
            self.predicted_probability = None
            self.predicted_class_label.configure(text="Predicted Class:")
            self.predicted_probability_label.configure(text="Predicted Probability:")
            
    def classify_image(self):
        # Check if an image is selected
        if self.image_path:
            # Load the image and resize it to 150*150 pixels
            image = Image.open(self.image_path)
            image = image.resize((150, 150))
            
            # Convert the image to a numpy array
            image_array = np.array(image)
            
            # Normalize the image array
            image_array = image_array / 255.0
            
            # Reshape the image array to match the input shape of the model
            image_array = image_array.reshape((1, 150, 150, 3))
            
            # Make a prediction using the loaded model
            predictions = model.predict(image_array)
            
            # Get the predicted class and probability
            predicted_class_index = np.argmax(predictions[0])
            predicted_class = self.class_names[predicted_class_index]
            predicted_probability = predictions[0][predicted_class_index]
            
            # Check if the predicted class belongs to recycle or trash material
            if predicted_class in ["cardboard", "glass", "metal", "paper", "plastic"]:
                predicted_material = "Recyclable"
            elif predicted_class == "trash":
                predicted_material = "Trash"
            else:
                predicted_material = "Unknown"
            
            # Update the predicted class label and probability label
            self.predicted_class_label.configure(text=f"Predicted Class: {predicted_class}")
            self.predicted_probability_label.configure(text=f"Predicted Probability: {predicted_probability:.2f}")
            self.predicted_material_label.configure(text=f"Predicted Material: {predicted_material}")
            
            # Update the predicted class and probability variables
            self.predicted_class = predicted_class
            self.predicted_probability = predicted_probability
                
            
            # Update the predicted class and probability labels
            self.predicted_class_label.configure(text="Predicted Class: {}".format(predicted_class))
            self.predicted_probability_label.configure(text="Predicted Probability: {:.2f}%".format(predicted_probability * 100))
            
            # Set the predicted class and probability
            self.predicted_class = predicted_class
            self.predicted_probability = predicted_probability
        else:
            # Clear the predicted class and probability labels if no image is selected
            self.predicted_class_label.configure(text="Predicted Class:")
            self.predicted_probability_label.configure(text="Predicted Probability:")
            self.predicted_class = None
            self.predicted_probability = None
            self.predicted_class_label.configure(text="Predicted Class:")
            self.predicted_probability_label.configure(text="Predicted Probability:")
            
    def clear_image(self):
        # Clear the image and predicted class labels
        self.image_label.configure(image=None)
        self.image_label.image = None
        self.predicted_class_label.configure(text="Predicted Class:")
        self.predicted_probability_label.configure(text="Predicted Probability:")
        self.predicted_class = None
        self.predicted_probability = None
        
        
# Run the GUI
if __name__ == "__main__":
    # Load the saved model
    model = load_model("model.keras")
    
    # Create the GUI
    gui = TrashRecycleGUI()
    
    # Start the GUI event loop
    gui.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
